In [1]:
import pandas as pd
from datetime import datetime, timezone, timedelta
import requests
from io import StringIO
import geohash2

## Extraction Tangara Stations

In [2]:
# Get start and nowcast Timestamp
def get_start_nowcast_timestamp(nowcast_datetime, start_datetime = None):
    #
    # Please nowcast_datetime must be: NOWCAST_DATETIME=$(TZ='America/Bogota' date '+%Y-%m-%dT%H:%M:%S')
    # Check the script bash: run.sh
    #

    # NowCast DateTime
    nowcast_datetime = datetime.fromisoformat(nowcast_datetime)
    print('-------------nowcast_datetime-------------->>>> ', nowcast_datetime)

    # Start DateTime
    if not start_datetime:
        # Start DateTime last 24 hours
        start_datetime = datetime.fromisoformat((nowcast_datetime - timedelta(hours=24)).isoformat())
    else:
        # Start DateTime
        start_datetime = datetime.fromisoformat(start_datetime)
    print('-------------start_datetime-------------->>>> ', start_datetime)

    # Current DateTime
    nowcast_timestamp = int(nowcast_datetime.timestamp() * 1000)
    # Start DateTime
    start_timestamp = int(start_datetime.timestamp() * 1000)

    print('-------------nowcast_timestamp-------------->>>> ', nowcast_timestamp)
    print('-------------start_timestamp-------------->>>> ', start_timestamp)

    return [start_timestamp, nowcast_timestamp]

In [3]:
# Time Period to Calculate NowCast AQI
#
# AQICN: https://aqicn.org/faq/2015-03-15/air-quality-nowcast-a-beginners-guide/
# AirNow: http://airnow.gov/
# NowCast: https://en.wikipedia.org/wiki/NowCast_(air_quality_index)
#
#
# Pipeline Parameters
parameters = catalog.load('parameters')
# Current DateTime
nowcast_datetime = parameters['nowcast_datetime']
# Start DateTime, optional
start_datetime = parameters['start_datetime']

# Get start and nowcast Timestamp
start_timestamp, nowcast_timestamp = get_start_nowcast_timestamp(nowcast_datetime, start_datetime)

print(
    ' nowcast_datetime:', nowcast_datetime, '\n',
    'nowcast_timestamp:', nowcast_timestamp, '\n',
    'start_datetime:', start_datetime, '\n',
    'start_timestamp:', start_timestamp
)

2022-09-29 18:59:25,156 - kedro.io.data_catalog - INFO - Loading data from `parameters` (MemoryDataSet)...
-------------nowcast_datetime-------------->>>>  2022-09-15 00:00:00-05:00
-------------start_datetime-------------->>>>  2022-09-14 00:00:00-05:00
-------------nowcast_timestamp-------------->>>>  1663218000000
-------------start_timestamp-------------->>>>  1663131600000
 nowcast_datetime: 2022-09-15T00:00:00-05:00 
 nowcast_timestamp: 1663218000000 
 start_datetime: None 
 start_timestamp: 1663131600000


In [4]:
# Request to InfluxDB API REST
def request_to_influxdb(sql_query):
    endpoint = "http://influxdb.canair.io:8086/query"
    database = "canairio"
    parameters = {
        'db': database,
        'q': sql_query,
        'epoch': 'ms'
    }
    # To get response as CSV text
    headers = {'Accept': 'application/csv'}
    # GET Request
    return requests.get(endpoint, params=parameters, headers=headers)

In [5]:
# Get SQL Query Tangara Stations
def get_sql_query_tangara_stations(start_datetime, end_datetime):
    sql_query = ""
    # Period DateTime
    period_time = f"time >= {start_datetime}ms AND time <= {end_datetime}ms"
    # SQL
    sql_query = "SELECT DISTINCT(geo) AS \"geohash\" "\
                "FROM \"fixed_stations_01\" WHERE "\
                    "(\"geo3\" = 'd29') AND "\
                    f"{period_time} "\
                "GROUP BY \"name\";"
    return sql_query

In [6]:
# Get SQL Query Data Measurement
def get_sql_query_measurement(tangaras, measurement, start_datetime, end_datetime):
    sql_query = ""
    # Period DateTime
    period_time = f"time >= {start_datetime}ms AND time <= {end_datetime}ms"
    # SQL
    for mac in tangaras['MAC'].to_list():
        sql_query += "SELECT \"name\", "\
                    f"last(\"{measurement}\") "\
                    "FROM \"fixed_stations_01\" WHERE "\
                    f"(\"name\" = '{mac}') AND "\
                    f"{period_time} " \
                    "GROUP BY time(30s) fill(none); "
    return sql_query[:-2]

In [7]:
# Get Data Frame Measurement
def get_df_measurement(tangaras, measurement, start_datetime, end_datetime):
    # Data Frame Sensors
    df_sensors = []
    # SQL Query Data Sensors
    sql_query = get_sql_query_measurement(tangaras, measurement, start_datetime, end_datetime)
    # InfluxDB API REST Request
    influxdb_api_request = request_to_influxdb(sql_query)
    #print(influxdb_api_request)
    df_influxdb_api_sensors = pd.read_csv(StringIO(influxdb_api_request.text), sep=",")

    # Remove/Add Columns
    df_influxdb_api_sensors = df_influxdb_api_sensors[['time', 'name.1', 'last']]
    df_influxdb_api_sensors.rename(columns={'time': 'DATETIME', 'name.1': 'MAC', 'last': measurement.upper()}, inplace=True)

    # Truncate Response
    for index, row in tangaras.iterrows():
        df_sensor = df_influxdb_api_sensors.loc[df_influxdb_api_sensors['MAC'] == row['MAC']].reset_index(drop=True)[['DATETIME', measurement.upper()]] # Warning
        if not df_sensor.empty:
            df_sensor.rename(columns={measurement.upper(): row['ID']}, inplace=True)
            df_sensor.set_index('DATETIME', inplace=True)
            df_sensors.append(df_sensor)
    
    # Join Data Frames
    df_sensors = df_sensors[0].join(df_sensors[1:]).reset_index()

    # Update datetime
    tz = timezone(timedelta(hours=-5))
    df_sensors['DATETIME'] = df_sensors['DATETIME'].apply(lambda x: datetime.fromtimestamp(int(x) / 1000, tz=tz).isoformat())
    
    # Update dtype
    df_sensors[df_sensors.columns.to_list()[1:]] = df_sensors[df_sensors.columns.to_list()[1:]].astype('float64')
    
    return df_sensors

In [8]:
# Get Data Frame Tangara Stations
def get_tangara_stations(start_datetime, end_datetime):
    # SQL Query Tangaras
    sql_query = get_sql_query_tangara_stations(start_datetime, end_datetime)
    # InfluxDB API REST Request
    influxdb_api_request = request_to_influxdb(sql_query)
    #print(influxdb_api_request)
    tangara_stations = pd.read_csv(StringIO(influxdb_api_request.text), sep=",")
    
    # Remove/Add Columns
    tangara_stations = tangara_stations[['tags', 'geohash']]
    tangara_stations['MAC'] = tangara_stations['tags'].apply(lambda x: x.split('=')[1])
    tangara_stations['GEOLOCATION'] = tangara_stations['geohash'].apply(lambda x: " ".join(f'{value:.8f}' for value in list(geohash2.decode_exactly(x)[0:2])))
    tangara_stations['LATITUDE'] = tangara_stations['GEOLOCATION'].apply(lambda x: x.split(' ')[0])
    tangara_stations['LONGITUDE'] = tangara_stations['GEOLOCATION'].apply(lambda x: x.split(' ')[1])
    tangara_stations['tags'] = tangara_stations['tags'].apply(lambda x: f"TANGARA_{x[-4:]}")
    tangara_stations.rename(columns={'tags': 'ID', 'geohash': 'GEOHASH'}, inplace=True)
    tangara_stations['GEOREGION'] = tangara_stations['GEOHASH'].apply(lambda x: x[:3])

    # Date time when query is executed
    tz = timezone(timedelta(hours=-5))
    tangara_stations['DATETIME'] = datetime.now(tz=tz).isoformat()

    # Reorder Columns
    tangara_stations = tangara_stations[['DATETIME', 'ID', 'MAC', 'GEOHASH', 'GEOREGION', 'GEOLOCATION', 'LATITUDE', 'LONGITUDE']]
    
    # Update dtype
    tangara_stations[['LATITUDE', 'LONGITUDE']] = tangara_stations[['LATITUDE', 'LONGITUDE']].astype('float64')

    return tangara_stations

In [9]:
# Data Frame Tangara Stations
tangara_stations = get_tangara_stations(start_timestamp, nowcast_timestamp)
tangara_stations.head()

,DATETIME,ID,MAC,GEOHASH,GEOREGION,GEOLOCATION,LATITUDE,LONGITUDE
0,2022-09-29T18:59:31.060440-05:00,TANGARA_2BBA,D29ESP32DE02BBA,d29e6b4,d29,3.38447571 -76.51634216,3.384476,-76.516342
1,2022-09-29T18:59:31.060440-05:00,TANGARA_14D6,D29ESP32DED14D6,d29dfx4,d29,3.33503723 -76.52732849,3.335037,-76.527328
2,2022-09-29T18:59:31.060440-05:00,TANGARA_1CE2,D29ESP32DED1CE2,d29e4cv,d29,3.35014343 -76.51222229,3.350143,-76.512222
3,2022-09-29T18:59:31.060440-05:00,TANGARA_1FCA,D29ESP32DED1FCA,d29e48s,d29,3.34327698 -76.52458191,3.343277,-76.524582
4,2022-09-29T18:59:31.060440-05:00,TANGARA_2492,D29ESP32DED2492,d29e64g,d29,3.39958191 -76.54792786,3.399582,-76.547928


In [9]:
# Check Data Types
tangara_stations.dtypes

DATETIME        object
ID              object
MAC             object
GEOHASH         object
GEOLOCATION     object
LATITUDE       float64
LONGITUDE      float64
dtype: object

In [10]:
# Describe Data
tangara_stations.describe()

,LATITUDE,LONGITUDE
count,13.000000,13.000000
mean,3.408878,-76.527328
std,0.046642,0.016191
min,3.335037,-76.547928
25%,3.384476,-76.542435
50%,3.399582,-76.527328
75%,3.446274,-76.516342
max,3.484726,-76.494370


In [11]:
# Save Data Frame into Catalog
catalog.save('tangara_stations', tangara_stations)

2022-09-21 22:26:07,956 - kedro.io.data_catalog - INFO - Saving data to `tangara_stations` (CSVDataSet)...


## Extraction - PM25 Raw

In [12]:
# SQL Query Data Measurement
sql_query = get_sql_query_measurement(tangara_stations, 'pm25', start_timestamp, nowcast_timestamp)
# print(sql_query)

In [13]:
# Data Frame PM25 Raw
pm25_raw = get_df_measurement(tangara_stations, 'pm25', start_timestamp, nowcast_timestamp)
pm25_raw.head()

,DATETIME,TANGARA_2BBA,TANGARA_14D6,TANGARA_1CE2,TANGARA_1FCA,TANGARA_2492,TANGARA_2FF6,TANGARA_48C6,TANGARA_4D7A,TANGARA_532E,TANGARA_EA06,TANGARA_F1AE,TANGARA_FAC6,TANGARA_06BE
0,2022-09-05T13:35:00-05:00,21.0,20.0,24.0,26.0,25.0,26.0,16.0,7.0,21.0,34.0,26.0,25.0,25.0
1,2022-09-05T13:35:30-05:00,20.0,20.0,23.0,25.0,24.0,25.0,15.0,7.0,19.0,42.0,26.0,25.0,26.0
2,2022-09-05T13:36:00-05:00,20.0,20.0,24.0,166.0,23.0,26.0,16.0,8.0,18.0,39.0,27.0,25.0,24.0
3,2022-09-05T13:36:30-05:00,21.0,20.0,24.0,64.0,25.0,24.0,16.0,8.0,19.0,36.0,29.0,25.0,24.0
4,2022-09-05T13:37:00-05:00,22.0,22.0,24.0,49.0,26.0,26.0,14.0,7.0,19.0,39.0,29.0,26.0,23.0


In [14]:
# Check Data Types
pm25_raw.dtypes

DATETIME         object
TANGARA_2BBA    float64
TANGARA_14D6    float64
TANGARA_1CE2    float64
TANGARA_1FCA    float64
TANGARA_2492    float64
TANGARA_2FF6    float64
TANGARA_48C6    float64
TANGARA_4D7A    float64
TANGARA_532E    float64
TANGARA_EA06    float64
TANGARA_F1AE    float64
TANGARA_FAC6    float64
TANGARA_06BE    float64
dtype: object

In [15]:
# Describe Data
pm25_raw.describe()

,TANGARA_2BBA,TANGARA_14D6,TANGARA_1CE2,TANGARA_1FCA,TANGARA_2492,TANGARA_2FF6,TANGARA_48C6,TANGARA_4D7A,TANGARA_532E,TANGARA_EA06,TANGARA_F1AE,TANGARA_FAC6,TANGARA_06BE
count,2838.000000,2747.000000,2796.000000,842.000000,2779.000000,2801.000000,2807.000000,2763.000000,2692.000000,2755.000000,2770.000000,2739.000000,2793.000000
mean,21.417195,24.116491,25.281474,38.681710,27.880533,26.601928,16.470609,5.638075,16.235884,32.560436,28.472924,21.766338,25.704261
std,5.746803,5.982577,6.981280,59.070996,11.151621,5.989012,6.511554,2.852142,8.509186,20.869400,12.251082,15.567362,5.954451
min,10.000000,12.000000,12.000000,23.000000,13.000000,14.000000,0.000000,2.000000,3.000000,9.000000,11.000000,9.000000,15.000000
25%,17.000000,20.000000,21.000000,27.000000,23.000000,22.000000,11.000000,3.000000,8.000000,15.000000,17.000000,11.000000,21.000000
50%,21.000000,23.000000,25.000000,29.000000,26.000000,26.000000,17.000000,5.000000,15.000000,29.000000,29.000000,20.000000,25.000000
75%,25.000000,28.000000,30.000000,31.000000,31.000000,30.000000,21.000000,8.000000,24.000000,48.000000,39.000000,31.000000,30.000000
max,44.000000,72.000000,56.000000,1110.000000,182.000000,42.000000,60.000000,12.000000,86.000000,583.000000,170.000000,449.000000,51.000000


In [16]:
# Save Data Frame into Catalog
catalog.save('pm25_raw', pm25_raw)

2022-09-21 22:26:11,089 - kedro.io.data_catalog - INFO - Saving data to `pm25_raw` (CSVDataSet)...


---

## Extraction - Temp Raw

In [17]:
# SQL Query Data Measurement
sql_query = get_sql_query_measurement(tangara_stations, 'tmp', start_timestamp, nowcast_timestamp)
# print(sql_query)

In [18]:
# Data Frame Temp Raw
temp_raw = get_df_measurement(tangara_stations, 'tmp', start_timestamp, nowcast_timestamp)
temp_raw.head()

,DATETIME,TANGARA_2BBA,TANGARA_14D6,TANGARA_1CE2,TANGARA_1FCA,TANGARA_2492,TANGARA_2FF6,TANGARA_48C6,TANGARA_4D7A,TANGARA_532E,TANGARA_EA06,TANGARA_F1AE,TANGARA_FAC6,TANGARA_06BE
0,2022-09-05T13:35:00-05:00,33.74,30.60,32.23,30.87,33.07,32.86,33.2,33.05,0.0,33.30,35.30,35.35,33.22
1,2022-09-05T13:35:30-05:00,33.80,30.55,32.26,30.86,33.05,32.86,33.2,33.05,0.0,33.29,35.28,35.37,33.23
2,2022-09-05T13:36:00-05:00,33.89,30.59,32.31,30.90,33.07,32.89,33.3,33.04,0.0,33.34,35.29,35.39,33.24
3,2022-09-05T13:36:30-05:00,33.99,30.61,32.45,30.91,33.08,32.98,33.3,33.04,0.0,33.39,35.28,35.36,33.23
4,2022-09-05T13:37:00-05:00,34.10,30.61,32.45,30.92,33.05,33.01,33.4,33.07,0.0,33.37,35.25,35.43,33.29


In [19]:
# Check Data Types
temp_raw.dtypes

DATETIME         object
TANGARA_2BBA    float64
TANGARA_14D6    float64
TANGARA_1CE2    float64
TANGARA_1FCA    float64
TANGARA_2492    float64
TANGARA_2FF6    float64
TANGARA_48C6    float64
TANGARA_4D7A    float64
TANGARA_532E    float64
TANGARA_EA06    float64
TANGARA_F1AE    float64
TANGARA_FAC6    float64
TANGARA_06BE    float64
dtype: object

In [20]:
# Describe Data
temp_raw.describe()

,TANGARA_2BBA,TANGARA_14D6,TANGARA_1CE2,TANGARA_1FCA,TANGARA_2492,TANGARA_2FF6,TANGARA_48C6,TANGARA_4D7A,TANGARA_532E,TANGARA_EA06,TANGARA_F1AE,TANGARA_FAC6,TANGARA_06BE
count,2838.000000,2747.000000,2796.000000,842.000000,2779.000000,2801.000000,2807.000000,2763.000000,2692.0,2755.000000,2770.000000,2739.000000,2793.000000
mean,30.240236,28.930200,28.751513,30.536401,31.692918,30.167072,30.200819,31.922617,0.0,29.610613,31.464451,31.459273,30.686889
std,4.053270,1.802855,3.049627,1.327256,2.339783,2.727612,3.287388,1.388718,0.0,3.818666,2.739840,4.073062,2.328387
min,24.600000,25.940000,24.540000,28.090000,28.250000,26.570000,25.100000,29.890000,0.0,25.120000,27.730000,26.630000,26.930000
25%,26.660000,27.220000,26.060000,29.430000,30.010000,27.430000,27.500000,30.710000,0.0,26.490000,28.750000,28.120000,28.560000
50%,29.170000,29.030000,28.095000,30.960000,31.820000,29.510000,29.100000,31.770000,0.0,27.690000,31.465000,29.400000,30.600000
75%,34.340000,30.340000,31.390000,31.720000,32.870000,32.750000,33.100000,33.070000,0.0,33.475000,34.250000,35.490000,32.780000
max,38.290000,32.130000,34.890000,32.700000,37.780000,35.270000,37.900000,34.470000,0.0,37.280000,36.250000,39.650000,34.790000


In [21]:
# Save Data Frame into Catalog
catalog.save('temp_raw', temp_raw)

2022-09-21 22:26:14,289 - kedro.io.data_catalog - INFO - Saving data to `temp_raw` (CSVDataSet)...


---

## Extraction - Hum Raw

In [22]:
# SQL Query Data Measurement
sql_query = get_sql_query_measurement(tangara_stations, 'hum', start_timestamp, nowcast_timestamp)
# print(sql_query)

In [23]:
# Data Frame Hum Raw
hum_raw = get_df_measurement(tangara_stations, 'hum', start_timestamp, nowcast_timestamp)
hum_raw.head()

,DATETIME,TANGARA_2BBA,TANGARA_14D6,TANGARA_1CE2,TANGARA_1FCA,TANGARA_2492,TANGARA_2FF6,TANGARA_48C6,TANGARA_4D7A,TANGARA_532E,TANGARA_EA06,TANGARA_F1AE,TANGARA_FAC6,TANGARA_06BE
0,2022-09-05T13:35:00-05:00,0.0,50.66,46.74,46.71,43.14,45.35,51.1,45.60,0.0,38.22,38.62,37.81,42.98
1,2022-09-05T13:35:30-05:00,0.0,50.72,46.89,46.96,43.14,45.33,51.0,45.89,0.0,38.19,38.63,37.75,42.86
2,2022-09-05T13:36:00-05:00,0.0,50.74,46.68,47.85,43.36,45.23,50.9,45.75,0.0,38.23,38.74,37.74,43.54
3,2022-09-05T13:36:30-05:00,0.0,50.67,47.09,47.66,43.58,45.56,50.8,45.75,0.0,38.11,38.70,37.77,40.57
4,2022-09-05T13:37:00-05:00,0.0,50.68,47.09,47.61,43.28,44.93,50.8,45.38,0.0,38.11,38.89,37.76,41.65


In [24]:
# Check Data Types
hum_raw.dtypes

DATETIME         object
TANGARA_2BBA    float64
TANGARA_14D6    float64
TANGARA_1CE2    float64
TANGARA_1FCA    float64
TANGARA_2492    float64
TANGARA_2FF6    float64
TANGARA_48C6    float64
TANGARA_4D7A    float64
TANGARA_532E    float64
TANGARA_EA06    float64
TANGARA_F1AE    float64
TANGARA_FAC6    float64
TANGARA_06BE    float64
dtype: object

In [25]:
# Describe Data
hum_raw.describe()

,TANGARA_2BBA,TANGARA_14D6,TANGARA_1CE2,TANGARA_1FCA,TANGARA_2492,TANGARA_2FF6,TANGARA_48C6,TANGARA_4D7A,TANGARA_532E,TANGARA_EA06,TANGARA_F1AE,TANGARA_FAC6,TANGARA_06BE
count,2838.0,2747.000000,2796.000000,842.000000,2779.000000,2801.000000,2807.000000,2763.000000,2692.0,2755.000000,2770.000000,2739.000000,2793.000000
mean,0.0,57.198460,57.591388,49.099394,47.671925,53.878322,63.328963,49.962486,0.0,50.105060,48.429809,48.375294,49.312854
std,0.0,5.905023,9.523785,4.939122,4.576806,7.479805,12.036714,3.129195,0.0,9.764171,7.653289,8.894346,6.435545
min,0.0,46.750000,39.920000,43.460000,35.670000,41.890000,43.100000,43.440000,0.0,32.760000,36.310000,32.660000,38.870000
25%,0.0,53.340000,49.157500,45.370000,44.990000,45.810000,51.700000,47.045000,0.0,38.480000,40.762500,37.855000,43.030000
50%,0.0,56.530000,59.820000,46.705000,48.080000,55.650000,63.900000,50.310000,0.0,54.760000,46.825000,52.740000,49.730000
75%,0.0,62.850000,66.942500,52.335000,51.110000,61.170000,73.500000,52.650000,0.0,58.510000,55.737500,56.010000,55.560000
max,0.0,66.360000,70.640000,60.000000,56.240000,65.810000,85.400000,54.260000,0.0,62.370000,60.750000,59.320000,61.320000


In [26]:
# Save Data Frame into Catalog
catalog.save('hum_raw', hum_raw)

2022-09-21 22:26:18,008 - kedro.io.data_catalog - INFO - Saving data to `hum_raw` (CSVDataSet)...


---

## Extraction - CO2 Raw

In [27]:
# SQL Query Data Measurement
sql_query = get_sql_query_measurement(tangara_stations, 'co2', start_timestamp, nowcast_timestamp)
# print(sql_query)

In [28]:
# Data Frame CO2 Raw
co2_raw = get_df_measurement(tangara_stations, 'co2', start_timestamp, nowcast_timestamp)
co2_raw.head()

,DATETIME,TANGARA_2BBA,TANGARA_14D6,TANGARA_1CE2,TANGARA_1FCA,TANGARA_2492,TANGARA_2FF6,TANGARA_48C6,TANGARA_4D7A,TANGARA_532E,TANGARA_EA06,TANGARA_F1AE,TANGARA_FAC6,TANGARA_06BE
0,2022-09-05T13:35:00-05:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,449.0,338.0,171.0
1,2022-09-05T13:35:30-05:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,445.0,345.0,172.0
2,2022-09-05T13:36:00-05:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,447.0,346.0,172.0
3,2022-09-05T13:36:30-05:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,451.0,345.0,170.0
4,2022-09-05T13:37:00-05:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,458.0,344.0,169.0


In [29]:
# Check Data Types
co2_raw.dtypes

DATETIME         object
TANGARA_2BBA    float64
TANGARA_14D6    float64
TANGARA_1CE2    float64
TANGARA_1FCA    float64
TANGARA_2492    float64
TANGARA_2FF6    float64
TANGARA_48C6    float64
TANGARA_4D7A    float64
TANGARA_532E    float64
TANGARA_EA06    float64
TANGARA_F1AE    float64
TANGARA_FAC6    float64
TANGARA_06BE    float64
dtype: object

In [30]:
# Describe Data
co2_raw.describe()

,TANGARA_2BBA,TANGARA_14D6,TANGARA_1CE2,TANGARA_1FCA,TANGARA_2492,TANGARA_2FF6,TANGARA_48C6,TANGARA_4D7A,TANGARA_532E,TANGARA_EA06,TANGARA_F1AE,TANGARA_FAC6,TANGARA_06BE
count,2838.0,2747.0,2796.0,842.0,2779.0,2801.0,2807.0,2763.0,2692.0,2755.0,2770.000000,2739.000000,2793.000000
mean,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,398.905054,334.679445,198.188686
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,38.390384,22.251702,21.817437
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,357.000000,300.000000,55.000000
25%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,375.000000,315.000000,185.000000
50%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,387.000000,332.000000,199.000000
75%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,410.000000,349.000000,210.000000
max,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,755.000000,443.000000,277.000000


In [31]:
# Save Data Frame into Catalog
catalog.save('co2_raw', co2_raw)

2022-09-21 22:26:21,316 - kedro.io.data_catalog - INFO - Saving data to `co2_raw` (CSVDataSet)...


---

## Extraction - X Raw